# Pick Management Using Market Basket Analysis

#### What is Market Basket Analysis? 
Market Basket analysis is one of the techniques used by retailers to identify the associations between items which are more
frequently bought together. This allows retailers to organize the selves in the store. It also helps develop marketing strategies, promotions, discounts etc.  

#### How it can be used for Pick Mangement?
In a warehouse to optimize picking process, the items which are frequently bought together can be kept in the same location/area. This will allow pickers to pick items in one go helping better utilize the resources and reduce the carbon foot print.

#### How is Association different than Recommendation?
* “Frequently Bought Together” → Association
* “Customers who bought this item also bought” → Recommendation

#### How Accociation Analysis Works?
Association rules are normally written like this: {Milk} -> {Egg} which means that there is a strong possiblity that a customers who purchased Milk will also purchas Egg in same transaction.

#### Frequently Used Terms
* Antecedent: In the above example, Milk is an antecedent.
* Consequent: In above exampple, Egg is a consequent.
* Support: Indicates how frequently an item or combination of items is purchased.

        Support = Freq(A,B)/N,   where N is the number of transactions
    
 
* Confidence: Likelihood that customer who bought A also bought B. Its divides the number of transactions involving both A and B by the number of transactions involving B.

        Confidence(A => B) = Freq(A,B)/Freq(A),  indicates proportion of transactions with item A, where B was also bought
        
  Confidence is also known as measure of the reliability of the rule. A confidence of .5 in the above example would mean that in 50% of the cases where Milk is bought, the Egg was also purchased.
  
  
* Lift: It indicates how likely an item B is purchased when item A is purchased, while controlling how popular item B is.
        
        Lift(A => B) = Support(A,B)/(Support(A) * Support(B))

      * Lift (A => B) = 1 means that there is no correlation within the itemset.
      * Lift (A => B) > 1 means that there is a positive correlation within the itemset, i.e., products in the itemset, A, and B, are more likely to be bought together.
      * Lift (A => B) < 1 means that there is a negative correlation within the itemset, i.e., products in itemset, A, and B, are unlikely to be bought together.  


In [1]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
# read the file
retail_data = pd.read_csv("online_retail.csv", encoding="ISO-8859-1")
retail_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


In [3]:
# Select features that could be made available to warhouse operations.
warehouse = retail_data[["InvoiceNo", "StockCode", "Description", "Quantity", "UnitPrice"]]
warehouse.head()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55
1,536365,71053,WHITE METAL LANTERN,6,3.39
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39


In [4]:
# Check for missing values
warehouse.isnull().sum()

InvoiceNo         0
StockCode         0
Description    1454
Quantity          0
UnitPrice         0
dtype: int64

In [5]:
# Desciption is important hence if not available, drop records with missing descrption
warehouse = warehouse[~warehouse["Description"].isnull()]

In [6]:
# removing records where quntity is 0 or -ve
warehouse = warehouse[warehouse["Quantity"] > 0]

In [7]:
# removing records where unit price is 0 or -ve
warehouse = warehouse[warehouse["UnitPrice"] > 0]

In [8]:
warehouse['Description'] = warehouse['Description'].str.strip()

In [9]:
warehouse.shape

(530104, 5)

In [10]:
# Check the structure of dataframe
warehouse.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 530104 entries, 0 to 541908
Data columns (total 5 columns):
InvoiceNo      530104 non-null object
StockCode      530104 non-null object
Description    530104 non-null object
Quantity       530104 non-null int64
UnitPrice      530104 non-null float64
dtypes: float64(1), int64(1), object(3)
memory usage: 24.3+ MB


In [11]:
basket = warehouse.groupby(["InvoiceNo","Description"])["Quantity"].sum().unstack().reset_index().fillna(0).set_index("InvoiceNo")
basket.head()

Description,*Boombox Ipod Classic,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
def encoder (x):
    if x>0:
        return 1
    else:
        return 0

basket_set =  basket.applymap(encoder) 

#for feature in [i for i in basket.columns if i != "InvoiceNo"]:
#    basket[feature] = basket[feature].apply(lambda x: 1 if x>0 else 0)

In [13]:
basket.head()

Description,*Boombox Ipod Classic,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Frequent Itemset Generation

In [14]:
# Find all frequent item-sets with support >= pre-determined min_support count
frequent_itemsets = apriori(basket_set, min_support=0.02, use_colnames=True)

### Rule Generation

In [17]:
# List all Association Rules from frequent item-sets. 
# Calculate Support and Confidence for all rules. 
# Prune rules that fail min_support and min_confidence thresholds.

#rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
rules.head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(60 TEATIME FAIRY CAKE CASES),(PACK OF 72 RETROSPOT CAKE CASES),0.041483,0.066132,0.022695,0.547101,8.272837,0.019952,2.061980
1,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE GREEN),0.039228,0.049098,0.020942,0.533844,10.872990,0.019016,2.039880
2,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.052655,0.049098,0.032064,0.608944,12.402571,0.029479,2.431625
3,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.049098,0.052655,0.032064,0.653061,12.402571,0.029479,2.730582
4,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE RED),0.039228,0.052655,0.023547,0.600255,11.399713,0.021482,2.369875
5,(CHARLOTTE BAG PINK POLKADOT),(CHARLOTTE BAG SUKI DESIGN),0.037174,0.044188,0.020541,0.552561,12.504660,0.018898,2.136181
6,(RED RETROSPOT CHARLOTTE BAG),(CHARLOTTE BAG PINK POLKADOT),0.051703,0.037174,0.026152,0.505814,13.606532,0.024230,1.948306
7,(CHARLOTTE BAG PINK POLKADOT),(RED RETROSPOT CHARLOTTE BAG),0.037174,0.051703,0.026152,0.703504,13.606532,0.024230,3.198346
8,(CHARLOTTE BAG SUKI DESIGN),(RED RETROSPOT CHARLOTTE BAG),0.044188,0.051703,0.025401,0.574830,11.117835,0.023116,2.230394
9,(STRAWBERRY CHARLOTTE BAG),(CHARLOTTE BAG SUKI DESIGN),0.036172,0.044188,0.020792,0.574792,13.007770,0.019193,2.247870


Lets take an example and upderstand what the rules says.

The support value for the first rule is 0.022. This number is calculated by dividing the number of transactions containing ‘60 TEATIME FAIRY CAKE CASES’ and ‘PACK OF 72 RETROSPOT CAKE CASES’ by the total number of transactions.

The confidence level for the rule is 0.547, which shows that out of all the transactions that contain ‘60 TEATIME FAIRY CAKE CASES’, 54.7 percent contain ‘PACK OF 72 RETROSPOT CAKE CASES’ too.

The lift of 8.27 tells us that ‘PACK OF 72 RETROSPOT CAKE CASES’ is 8.27 times more likely to be bought by the customers who buys ‘60 TEATIME FAIRY CAKE CASES’ compared to the default likelihood sale of ‘PACK OF 72 RETROSPOT CAKE CASES’.

Sources:

    https://www.kdnuggets.com/2019/12/market-basket-analysis.html
    
    https://pbpython.com/market-basket-analysis.html